In [1]:
import os
os.chdir("../")

In [2]:
import wandb
import torch
import random
import time
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
from torch import optim
from tqdm import tqdm
from models.vt_resnet20 import VTResNet20
from torch.utils.data import DataLoader, Subset
from typing import Any
from torchsummary import summary

from torchvision import datasets

In [3]:
%load_ext autoreload
%autoreload 2

# Utility Functions

In [4]:
def random_seed(seed):
    """Set seed"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

def store_params(content, name):
    f = open(f'params/{name}.pkl','wb')
    pickle.dump(content, f)
    f.close()

def load_params(name):
    fl = open(f'params/{name}.pkl', "rb")
    loaded = pickle.load(fl)
    return loaded

def store_model(model, name):
    torch.save(model.state_dict(), f'./trained_models/{name}.pth')
                                

# Data Preparation

In [5]:
project_name = 'vt_resnet'
cores = 12
random_seed(8)
input_dim = 32
batch_size = 128
num_classes = 10

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((input_dim, input_dim)),
    #transforms.RandomResizedCrop(input_dim),
    transforms.RandomHorizontalFlip(),
    normalize
])

valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((input_dim, input_dim)),
    normalize
])

train_dataset = datasets.CIFAR10(root='/datasets/cifar', train = True, transform = train_transform, download = False)
valid_dataset = datasets.CIFAR10(root='/datasets/cifar', train = False, transform = valid_transform, download = False)

In [7]:
len(train_dataset), len(valid_dataset)

(50000, 10000)

In [8]:
total_valid_num = len(valid_dataset)
total_train_num = len(train_dataset)
valid_num = int(0.5 * total_valid_num)

valid_mask = list(range(valid_num))
test_mask = list(range(valid_num, total_valid_num))

valid_loader = DataLoader(Subset(valid_dataset, valid_mask), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(Subset(valid_dataset, test_mask), batch_size=batch_size, shuffle=True)

small_train_mask = random.sample(range(total_train_num), 1200)
medium_train_mask = random.sample(range(total_train_num), 5000)
small_valid_mask = random.sample(range(total_valid_num), 200)

small_train_loader = DataLoader(Subset(train_dataset, list(small_train_mask)), batch_size=batch_size, 
                                shuffle=True, num_workers=2)
small_valid_loader = DataLoader(Subset(valid_dataset, list(small_valid_mask)), batch_size=batch_size, 
                                shuffle=True, num_workers=2)

medium_loader = DataLoader(Subset(train_dataset, list(medium_train_mask)), batch_size=batch_size, shuffle=True)

# Training

In [29]:
hyperparameters = {
    'epochs': 20,
    'vt_num_layers':3,
    'resnet_pretrained': True,
    'freeze_resnet': False,
    'batch_size': 128,
    'learning_rate': 0.0005,
    'vt_channels': 64,
    'transformer_enc_layers': 2,
    'transformer_n_heads': 1,
    'transformer_fc_dims': 128,
    'transformer_dropout': 0.5,
    'tokens': 4,
    'token_dims': 128,
    'optimizer': 'adam',
    'weight_decay': 8e-5,
    'input_dim': input_dim,
    'num_classes': num_classes,
}

In [25]:
def evaluate(model: nn.Module, data_loader: Any, device: torch.device, comment: str = ""):
    
    model.eval()
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0
    loss_history = []

    with torch.no_grad():
        for data, target in tqdm(data_loader):
            data = data.to(device)
            target = target.to(device)
            
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)

            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()
    
    avg_loss = total_loss / total_samples
    wandb.log({'valid_loss': avg_loss})
    
    accuracy = 100.0 * correct_samples / total_samples
    return accuracy

In [26]:
def train(model, optimizer, epochs, data_loader, test_loader, device):
    
    wandb.watch(model, log="all", log_freq=10)
    
    full_start = time.time()
    for i in range(epochs):
        
        model.train()
        model.to(device)
        print(f"Starting Epoch {i}")
        
        total_loss = 0
        epoch_time = time.time()
        num_batches = 0
        for j, (data, label) in enumerate(data_loader):
            
            data, label = data.to(device), label.to(device)
            optimizer.zero_grad()
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, label)
            loss.backward()
            
            total_loss += loss.item()
            optimizer.step()
            num_batches += 1
            
            wandb.log({'batch_loss': loss.item()})
        print(f"Finished Epoch {i}")
        
        valid_accuracy = evaluate(model, test_loader, device)
        train_accuracy = evaluate(model, data_loader, device)
        
        print(f"Validation Accuracy: ", valid_accuracy)
        print(f"Training Accuracy: ", train_accuracy)
        
        wandb.log({
            'loss': total_loss / num_batches,
            'valid_accuracy': valid_accuracy,
            'train_accuracy': train_accuracy,
            'epoch_time_minutes': (time.time() - epoch_time) / 60
        })
    wandb.log({'full_run_time_minutes': (time.time() - full_start) / 60})
        

In [27]:
def train_model(hyperparameters):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    
    with wandb.init(project=project_name, config=hyperparameters):
       
        config = wandb.config
        
        model = VTResNet20(
            vt_num_layers=config.vt_num_layers,
            tokens=config.tokens,
            token_channels=config.token_dims,
            input_dim=config.input_dim,
            vt_channels=config.vt_channels,
            transformer_enc_layers=config.transformer_enc_layers,
            transformer_heads=config.transformer_n_heads,
            transformer_fc_dim=config.transformer_fc_dims,
            transformer_dropout=config.transformer_dropout,
            num_classes=config.num_classes,
            resnet_pretrained=config.resnet_pretrained,
            freeze_resnet=config.freeze_resnet,
        )

        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
           
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=cores)    
        
        train(model, optimizer, config.epochs, train_loader, valid_loader, device)

        test_accuracy = evaluate(model, test_loader, device)
        
        sample = train_dataset[0][0].reshape(1, 3, config.input_dim, config.input_dim)
        
        wandb.log({'test_accuracy': test_accuracy})
    
    return model, test_accuracy

In [30]:
model, test_accuracy = train_model(hyperparameters)

cuda:0


Starting Epoch 0


  8%|▊         | 3/40 [00:00<00:01, 20.54it/s]

Finished Epoch 0


100%|██████████| 391/391 [00:09<00:00, 40.95it/s]

...

  8%|▊         | 3/40 [00:00<00:01, 20.41it/s]

Validation Accuracy:  tensor(86.5600, device='cuda:0')
Training Accuracy:  tensor(93.6440, device='cuda:0')


100%|██████████| 40/40 [00:01<00:00, 22.26it/s]


batch_loss,0.25741
_runtime,794
_timestamp,1615885155
_step,7882
valid_loss,0.45123
loss,0.20105
valid_accuracy,86.56
train_accuracy,93.644
epoch_time_minutes,0.66363
full_run_time_minutes,13.0903
test_accuracy,85.57999


batch_loss,█▄▄▄▃▂▃▃▂▃▃▃▂▂▂▂▂▃▂▂▂▁▂▂▂▁▂▂▁▂▂▂▁▂▂▂▁▂▁▂
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid_loss,█▇▆▅▅▄▅▃▅▃▅▃▄▃▄▂▄▂▄▂▄▂▄▂▅▂▄▁▄▁▄▁▄▁▄▁▄▁▄▄
loss,█▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
valid_accuracy,▁▄▆▆▆▆▇▇▇█▇▇▇███████
train_accuracy,▁▄▅▅▆▅▆▇▇▇▇▇▇██▇████
epoch_time_minutes,▄▅▅▄▇▃▆▁▃▃█▃▅▃▃█▄▅▂▇
full_run_time_minutes,▁
test_accuracy,▁


In [14]:
test_accuracy

tensor(84.1800, device='cuda:0')

In [15]:
# summary(model.to(torch.device('cuda:0')), (3, 224, 224))

RuntimeError: shape '[2, 64, 16, 16]' is invalid for input of size 1605632